In [45]:
import pandas as pd
import numpy as np
import math

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix

from sklearn.ensemble import RandomForestClassifier

import sklearn.cluster as cluster
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
#from sklearn.mixture import GMM

from sklearn.metrics import silhouette_samples, silhouette_score
from scipy.cluster.hierarchy import fcluster
from scipy.cluster.hierarchy import dendrogram, linkage




In [2]:
dataset = pd.read_csv("merged_training_dataset.csv")

In [3]:
dataset = dataset[dataset['order_status'] == 'delivered']
len(dataset)

115728

In [4]:
dataset.columns

Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date',
       'quantity', 'product_id', 'seller_id', 'shipping_limit_date',
       'product_price', 'freight_value', 'payment_sequential', 'payment_type',
       'payment_installments', 'payment_value', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm',
       'product_category', 'review_id', 'review_score', 'review_comment_title',
       'review_comment_message', 'review_creation_date',
       'review_answer_timestamp', 'seller_zip_code_prefix', 'seller_city',
       'seller_state', 'seller_lat', 'sellet_lng', 'customer_unique_id',
       'customer_zip_code_prefix', 'customer_city', 'customer_state',
       'customer_lat', 'customer_lng'],
      dtype='object')

In [5]:
dataset.dropna(subset=[ 'order_delivered_customer_date', 'order_delivered_carrier_date', 'order_approved_at', 'seller_lat', 'customer_lat','product_category'],inplace=True)
len(dataset)

113487

In [6]:
dataset.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,quantity,product_id,...,seller_city,seller_state,seller_lat,sellet_lng,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,customer_lat,customer_lng
0,00010242fe8c5a6d1ba2dd792cb16214,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13T08:59:02Z,2017-09-13T09:45:35Z,2017-09-19T18:34:16Z,2017-09-20T23:43:48Z,2017-09-29T00:00:00Z,1,4244733e06e7ecb4970a6e2683c13e61,...,volta redonda,SP,-22.498183,-44.123614,871766c5855e863f6eccc05f988b23cb,28013,campos dos goytacazes,RJ,-21.758076,-41.312633
1,00018f77f2f0320c557190d7a144bdd3,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26T10:53:06Z,2017-04-26T11:05:13Z,2017-05-04T14:35:00Z,2017-05-12T16:04:24Z,2017-05-15T00:00:00Z,1,e5f2d52b802189ee658865ca93d83a8f,...,sao paulo,SP,-23.566258,-46.518417,eb28e67c4c0b83846050ddfb8a35d051,15775,santa fe do sul,SP,-20.212393,-50.941471
2,000229ec398224ef6ca0657da4fc703e,6489ae5e4333f3693df5ad4372dab6d3,delivered,2018-01-14T14:33:31Z,2018-01-14T14:48:30Z,2018-01-16T12:36:48Z,2018-01-22T13:19:16Z,2018-02-05T00:00:00Z,1,c777355d18b72b67abbeef9df44fd0fd,...,borda da mata,MG,-22.264094,-46.158564,3818d81c6709e39d06b2738a8d3a2474,35661,para de minas,MG,-19.860439,-44.597972
3,00024acbcdf0a6daa1e931b038114c75,d4eb9395c8c0431ee92fce09860c5a06,delivered,2018-08-08T10:00:35Z,2018-08-08T10:10:18Z,2018-08-10T13:28:00Z,2018-08-14T13:32:39Z,2018-08-20T00:00:00Z,1,7634da152a4610f1595efa32f14722fc,...,franca,SP,-20.548228,-47.395897,af861d436cfc08b2c2ddefd0ba074622,12952,atibaia,SP,-23.144923,-46.539830
4,00042b26cf59d7ce69dfabb4e55b4fd9,58dbd0b2d70206bf40e62cd34e84d795,delivered,2017-02-04T13:57:51Z,2017-02-04T14:10:13Z,2017-02-16T09:46:09Z,2017-03-01T16:42:31Z,2017-03-17T00:00:00Z,1,ac6c3623068f30de03045865e4e10089,...,loanda,PR,-22.931427,-53.133759,64b576fb70d441e8f1b2d7d446e483c5,13226,varzea paulista,SP,-23.249008,-46.824961


In [7]:
dataset.order_purchase_timestamp = pd.to_datetime(dataset.order_purchase_timestamp)
#dataset.order_aproved_at = pd.to_datetime(dataset.order_aproved_at)
dataset.order_estimated_delivery_date = pd.to_datetime(dataset.order_estimated_delivery_date)
dataset.order_delivered_customer_date = pd.to_datetime(dataset.order_delivered_customer_date)
dataset.review_creation_date = pd.to_datetime(dataset.review_creation_date)
dataset.review_answer_timestamp = pd.to_datetime(dataset.review_answer_timestamp)
dataset.product_category.replace(['None', 'NaN', np.nan], "NIL", inplace=True)

In [8]:
n = len(dataset)
training_n = round(0.6 * n)
validation_n = round(0.2 * n)
train = dataset.iloc[0: training_n, ]

In [9]:
valid = dataset.iloc[training_n: training_n+validation_n, ]
test = dataset.iloc[training_n+validation_n: ,]

In [10]:
y_train = train['review_score'] >= 3 
y_valid = valid['review_score'] >= 3
y_test = test['review_score'] >= 3

# START

In [30]:
len(clusters)

55505

In [30]:
len(clusters)

55505

## K MEANS CLUSTERING FOR CUSTOMERS

In [38]:
def kclus_cust(train):
    recency_scores = train.groupby('customer_unique_id').order_purchase_timestamp.max().reset_index()
    recency_scores.columns = ['customer_unique_id', 'MaxPurchaseDate']
    recency_scores['Recency'] = (recency_scores['MaxPurchaseDate'].max() - recency_scores['MaxPurchaseDate']).dt.days
    frequency_scores = train.groupby('customer_unique_id').order_purchase_timestamp.count().reset_index()
    frequency_scores.columns = ['customer_unique_id','Frequency']
    monetory_scores = train.groupby('customer_unique_id').payment_value.sum().reset_index()
    monetory_scores.columns = ['customer_unique_id','Monetary']
    df_rfm = pd.merge(recency_scores, frequency_scores, on='customer_unique_id')
    df_rfm = pd.merge(df_rfm, monetory_scores, on='customer_unique_id')
    rfm_r_log = np.log(df_rfm['Recency']+0.1) #can't take log(0) and so add a small number
    rfm_f_log = np.log(df_rfm['Frequency'])
    rfm_m_log = np.log(df_rfm['Monetary']+0.1)
    log_data = pd.DataFrame({'Monetary': rfm_m_log,'Recency': rfm_r_log,'Frequency': rfm_f_log})
    matrix = log_data.as_matrix()
    kmeans = KMeans(init='k-means++', n_clusters = 2, n_init=100)
    kmeans.fit(matrix)
    clusters = kmeans.predict(matrix)
    return(clusters)


In [39]:
kclus_cust(train)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  from ipykernel import kernelapp as app


array([1, 1, 1, ..., 1, 1, 1], dtype=int32)

# Heirarchial clustering for products

In [46]:
def hei_prod(train):
    product_cat = train.groupby('product_category').order_purchase_timestamp.count().reset_index()
    product_cat.columns = ['product_category', 'Frequency']
    ms = train.groupby('product_category').payment_value.sum().reset_index()
    ms.columns = ['product_category','Monetary']
    df_prd = pd.merge(product_cat,ms, on='product_category')
    pd_f_log = np.log(df_prd ['Frequency'])
    pd_m_log = np.log(df_prd ['Monetary']+0.1)
    log_pd_data = pd.DataFrame({'Monetary': pd_m_log,'Frequency': pd_f_log})
    linked = linkage(log_pd_data, 'weighted')
    k=2
    l1=pd.DataFrame(fcluster(linked, k, criterion='maxclust'))
    k=3
    l2=pd.DataFrame(fcluster(linked, k, criterion='maxclust'))
    k=5
    level3=fcluster(linked, k, criterion='maxclust')
    k=7
    l3= pd.DataFrame(fcluster(linked, k, criterion='maxclust'))
    k=10
    l4= pd.DataFrame(fcluster(linked, k, criterion='maxclust'))
    TH = pd.DataFrame({'productcategory': list(df_prd['product_category']),'Level1': l1[0],'Level2': l2[0],'Level3': l3[0],'Level4': l4[0] })
    level_onehot = TH.copy()
    level_onehot = pd.get_dummies(level_onehot, columns=['Level1','Level2','Level3','Level4'])
    return level_onehot
    

In [47]:
hei_prod(train)

,productcategory,Level1_1,Level1_2,Level2_1,Level2_2,Level2_3,Level3_1,Level3_2,Level3_3,Level3_4,...,Level4_1,Level4_2,Level4_3,Level4_4,Level4_5,Level4_6,Level4_7,Level4_8,Level4_9,Level4_10
0,agro_industry_and_commerce,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,air_conditioning,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,art,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,arts_and_craftmanship,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,audio,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,stationery,1,0,1,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
67,tablets_printing_image,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
68,telephony,1,0,1,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
69,toys,1,0,1,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0


# END